In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf

from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.models import Model
from keras.preprocessing import text, sequence
from sklearn.model_selection import train_test_split

from clr_callback import *
from keras.optimizers import *

Using TensorFlow backend.


In [2]:
# Fix cudnn bug
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [3]:
colnames = ["product/productId",
            "review/userId",
            "review/helpfulness",
            "review/score",
            "review/time",
            "review/summary",
            "review/text"]
df = pd.read_csv("data/finemuged.csv", encoding='latin1', header=None,
                 names=colnames, quotechar="\"")

In [4]:
def one_hot(x, maxi=5):
    arr = [0]*maxi
    arr[x-1] = 1
    return arr

In [5]:
sequence_length = 256
vocabulary_size = 10000
embedding_dim = 300
filter_sizes = [3,4,5]
num_filters = 256
drop = 0.65

epochs = 3
batch_size = 2048

In [6]:
t = text.Tokenizer(vocabulary_size)
X = df["review/text"].values
t.fit_on_texts(X)
X = t.texts_to_sequences(X)
X = sequence.pad_sequences(X, value=0, padding='post', maxlen=sequence_length)
y = df["review/score"].astype(int).values.reshape(-1) - 1
y = np.eye(5)[y]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

In [8]:
embeddings_index = dict()
f = open('data/wiki-news-300d-1M.vec')
_ = f.readline()
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 999994 word vectors.


In [9]:
embedding_matrix = np.zeros((vocabulary_size, embedding_dim))
for word, i in t.word_index.items():
    if i >= vocabulary_size:
        break
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [10]:
inputs = Input(shape=(sequence_length,), dtype='int32')
embedding = Embedding(input_dim=vocabulary_size, output_dim=embedding_dim, weights=[embedding_matrix],
                      input_length=sequence_length)(inputs)
reshape = Reshape((sequence_length,embedding_dim,1))(embedding)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim),
                padding='valid', kernel_initializer='normal',
                activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim),
                padding='valid', kernel_initializer='normal',
                activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim),
                padding='valid', kernel_initializer='normal',
                activation='relu')(reshape)

maxpool_0 = MaxPool2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1),
                      strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(sequence_length - filter_sizes[1] + 1, 1),
                      strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(sequence_length - filter_sizes[2] + 1, 1),
                      strides=(1,1), padding='valid')(conv_2)

concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(5, activation='softmax')(dropout)

W0711 16:23:30.267900 139673741526848 deprecation_wrapper.py:119] From /home/jader/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0711 16:23:30.268748 139673741526848 deprecation_wrapper.py:119] From /home/jader/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0711 16:23:30.271547 139673741526848 deprecation_wrapper.py:119] From /home/jader/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0711 16:23:30.279748 139673741526848 deprecation_wrapper.py:119] From /home/jader/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [11]:
model = Model(inputs=inputs, outputs=output)
clr_triangular = CyclicLR(max_lr=0.01)

In [12]:
model.compile(optimizer=Adam(beta_1=0.9, beta_2=0.999, epsilon=1e-08),
              loss='binary_crossentropy', metrics=['accuracy'])

W0711 16:23:30.865796 139673741526848 deprecation_wrapper.py:119] From /home/jader/.local/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0711 16:23:30.871119 139673741526848 deprecation.py:323] From /home/jader/.local/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [13]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1,
          validation_data=(X_test, y_test), callbacks=[clr_triangular])

Train on 397917 samples, validate on 170537 samples
Epoch 1/3
397917/397917 [==============================] - 214s 538us/step - loss: 0.2862 - acc: 0.8885 - val_loss: 0.2298 - val_acc: 0.9086
Epoch 2/3
397917/397917 [==============================] - 209s 526us/step - loss: 0.2226 - acc: 0.9109 - val_loss: 0.2113 - val_acc: 0.9153
Epoch 3/3
397917/397917 [==============================] - 213s 534us/step - loss: 0.2053 - acc: 0.9181 - val_loss: 0.2018 - val_acc: 0.9203
